In [1]:
import os
import torch
import torch.nn as nn

    # A PyTorch implementation of a Multi-Layer Perceptron (MLP) model.

    # Attributes:
    #     layers (nn.ModuleList): A list of layers in the MLP, including both Linear layers and activation functions.

    # Methods:
    #     __init__(input_size, output_size, hidden_layers, activation_fn=nn.LeakyReLU):
    #         Initializes the MLP model with the specified input size, output size, hidden layers, and activation function.
    #     forward(x):
    #         Defines the forward pass of the MLP model.
    #     decode_model(model_path):
    #         Decodes the model architecture from the filename of a saved model.
    #     create_and_load(model_path, input_size, output_size):
    #         Creates an MLP model based on the architecture encoded in the filename and loads its weights from a file.

    # Usage:
    #     - This class is designed to create and manage MLP models with a flexible number of hidden layers and neurons.
    #     - It supports loading pre-trained models from a file.

    # Details:
    #     1. The number of hidden layers is limited to a maximum of 7 to prevent overly deep architectures.
    #     2. The activation function for the hidden layers can be customized (default is LeakyReLU).
    #     3. The model architecture can be encoded in the filename of a saved model, allowing for easy reconstruction.




class MLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, activation_fn=nn.LeakyReLU):

        super(MLP, self).__init__()
        if len(hidden_layers) > 7:
            raise ValueError("The number of hidden layers cannot exceed 7.")

        self.layers = nn.ModuleList()
        prev_size = input_size

        # Create hidden layers
        for neurons in hidden_layers:
            self.layers.append(nn.Linear(prev_size, neurons))
            self.layers.append(activation_fn())  # Add the specified activation function
            prev_size = neurons

        # Create output layer
        self.layers.append(nn.Linear(prev_size, output_size))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    @staticmethod
    def decode_model(model_path):
        model_path = os.path.basename(model_path)
        model = model_path.removesuffix(".pth")
        layers = model.split('_')
        layers.pop(0)
        return list(map(int, layers))

    @staticmethod
    def create_and_load(model_path, input_size, output_size):
        #print(f"Loading model from {model_path}")
        model = MLP(input_size, output_size, hidden_layers=MLP.decode_model(model_path))
        if torch.cuda.is_available():
            model.load_state_dict(torch.load(model_path))
        else:
            model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
        return model

